# Generate intraday bars using bars data service - F#

### Overview
Bars are widely used to illustrate price movements for an instrument over a look back period. Commonly used in financial analysis and trading strategies as per a deicion making technical indicator.<br>

This sample demonstrates how to directly request and plot `intraday bars` with a **customizable** time range sampling.<br>

It enables to retrieve **on-demand** data by calling a dedicated bars service on a tick by tick data store. 

### Inputs/outputs
Bars sample requires instrument's identifier, date time intervals as per inputs. It returns open-high-low-close (OHLC) and volume.
This sample also shows how to plot a simple graph for basis financial analysis using an open source library.

### Services used
This sample uses *gRPC requests* in order to retrieve bars from the dedicated hosted service. The queried endpoint in this script are:
* *TickBarsService*: to directly retrieve bars objects from the server

### Packages required
1. Systemathics packages:
    * *Systemathics.Apis.Type.Shared.V1*
    * *Systemathics.Apis.Services.TickAnalytics.V1*
2. Open source packages
    * *Google.Protobuf.WellKnownTypes*
    * *Google.Type*
    * *Grpc.Net.Client*
    * *Grpc.Core*
    * *XPlot.Plotly* as per display package

***

# Run bars sample

### Step 1: Install packages

In [1]:
#i "nuget: file://~/.nuget/packages/"
#i "nuget: https://api.nuget.org/v3/index.json"
#r "nuget: Systemathics.Apis, 0.*-pre*"
#r "nuget: XPlot.Plotly.Interactive"
#r "nuget: TaskBuilder.fs"

The below script needs to be able to find the current output cell; this is an easy method to get it.

Restore sources file://~/.nuget/packages/ https://api.nuget.org/v3/index.json Installed Packages Systemathics.Apis, 0.13.2-pre TaskBuilder.fs, 2.1.0 XPlot.Plotly.Interactive, 4.0.6

Loading extensions from `XPlot.Plotly.Interactive.dll`

Configuring PowerShell Kernel for XPlot.Plotly integration.

Installed support for XPlot.Plotly.

In [2]:
open Systemathics.Apis.Helpers
open Systemathics.Apis.Type.Shared.V1
open Systemathics.Apis.Services.TickAnalytics.V1
open Google.Protobuf.WellKnownTypes
open Google.Type
open Grpc.Net.Client
open Grpc.Core
open XPlot.Plotly
open XPlot.Plotly.Interactive

### Step 2: Prepare API requests
The following code snippets retrieve authentication token and prepare the API request by: opening the *channel* and adding the *token* to the request header:

In [3]:
// Get token as metadata
let headers = TokenHelpers.GetTokenAsMetaData();

// Create communication channel
let channel = ChannelHelpers.GetChannel();

### Step 3: Create and process request
To request *tick bars service*, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates
* Bars request parameters

#### 3.1 Instrument selection

In [4]:
let ticker = "AAPL"
let exchange = "BATS"

#### 3.2 Bars calculation parameters

In [5]:
// Set the bar duration
let sampling = 5L * 60L

// Set the bar calculation field
let field = BarPrice.Trade 

#### 3.3 Time period delimitation

In [6]:
// Create time intervals
let yesterday = DateTime.Today.AddDays(-1.)
let start = yesterday.AddDays(-50.)

// Build the bars request date interval (we are using Google date time format : we have to cast the dates)
let dateIntervals = 
  new DateInterval(
    StartDate = new Date ( Year = start.Year, Month = start.Month, Day = start.Day ),
    EndDate = new Date ( Year = yesterday.Year, Month = yesterday.Month, Day = yesterday.Day )
  )

// Build the bars request time interval (we are using Google date time format : we have to cast the dates)
// UTC time zone
let timeInterval = 
  new TimeInterval(
    StartTime = new TimeOfDay ( Hours = 15, Minutes = 00, Seconds = 00 ),
    EndTime = new TimeOfDay ( Hours = 16, Minutes = 00, Seconds = 00 ) 
  )

#### 3.4 Request creation
The following code snippet creates *gRPC client*, process request and returns the request reply:

In [7]:
// Generate constraints based on the previous space selection:
let constraints = new Constraints() 
constraints.DateIntervals.Add(dateIntervals)
constraints.TimeIntervals.Add(timeInterval)

In [8]:
// Generate intraday bars request
let request = 
    new TickBarsRequest(
        Identifier = new Identifier ( Exchange = exchange, Ticker = ticker), 
        Constraints = constraints,  
        Sampling = new Duration (Seconds = sampling),
        Field = field
    )

In [9]:
// Instantiate the intraday bars service
let service = new TickBarsService.TickBarsServiceClient(channel)

In [10]:
// Process intraday bars request
open FSharp.Control.Tasks.V2
open System.Collections.Generic

let bars = new List<TickBarsResponse>()

let iterAsync (t: IAsyncEnumerable<'T>) (f : 'T -> unit) =
    let task = 
        task {
            let e = t.GetAsyncEnumerator()
            let mutable go = true
            try
                let! step = e.MoveNextAsync()
                go <- step
                while go do 
                    f e.Current
                    let! step = e.MoveNextAsync()
                    go <- step
            finally
                e.DisposeAsync().AsTask().Wait()
        }
    task.Wait()
    
let call = service.TickBars(request, headers)
let e = call.ResponseStream.ReadAllAsync()

iterAsync e bars.Add


In [11]:
// Calculate the total ticks inside all the bars we received
let ticks = bars |> Seq.sumBy (fun bar -> bar.Count)

// Display
Console.WriteLine($"Fetched {bars.Count:N0} bars from (Total trades={ticks:N0})")
display(bars)

Fetched 72 bars from (Total trades=36,201)


index TimeStamp Open High Low Close Volume Count Vwap 0 Seconds Nanos 1662476400 0 156.14 156.14 155.75 155.83 49054 690 155.90354537856228 1 Seconds Nanos 1662476700 0 155.82 156.52 155.71 156.25 59009 776 156.13603407954722 2 Seconds Nanos 1662477000 0 156.28 156.34 155.87 155.97 30035 428 156.06097386382544 3 Seconds Nanos 1662477300 0 156.01 156.04 155.66 155.71 36753 636 155.80791431991938 4 Seconds Nanos 1662477600 0 155.7 156.06 155.57 155.99 59571 522 155.73472688052917 5 Seconds Nanos 1662477900 0 155.99 156.15 155.485 155.58 38637 580 155.78379597277228 6 Seconds Nanos 1662478200 0 155.585 156.35 155.48 155.99 47321 710 155.88639959003396 7 Seconds Nanos 1662478500 0 156.06 156.1 155.58 155.79 37659 573 155.79888884463196 8 Seconds Nanos 1662478800 0 155.75 156.45 155.72 156.06 35609 579 156.1537507371734 9 Seconds Nanos 1662479100 0 156.07 156.19 155.73 155.76 25739 393 155.93557034072808 10 Seconds Nanos 1662479400 0 155.76 156.19 155.71 156.06 25825 432 155.98441200387234 11 Seconds Nanos 1662479700 0 156.02 156.27 155.97 156.07 24166 417 156.14830050484156 12 Seconds Nanos 1662649200 0 155.76 156.13 155.76 156.03 55373 734 155.95811180539232 13 Seconds Nanos 1662649500 0 156.03 156.26 155.98 156.18 33564 412 156.13940576212616 14 Seconds Nanos 1662649800 0 156.19 156.2 155.93 156.14 40067 437 156.04044762522773 15 Seconds Nanos 1662650100 0 156.15 156.24 155.96 156.18 28876 364 156.09069763817703 16 Seconds Nanos 1662650400 0 156.19 156.35 156.105 156.16 31987 387 156.20363772782702 17 Seconds Nanos 1662650700 0 156.18 156.19 155.92 155.98 43564 457 156.05151019190146 18 Seconds Nanos 1662651000 0 155.985 156.17 155.91 156.14 30041 401 156.0437220798241 19 Seconds Nanos 1662651300 0 156.12 156.18 155.66 155.68 40910 504 155.9594530677096 (52 more)

### Step 4: Visualize data

#### 4.1 Plot the bars Chart
Plot the bars request result and plot them with the package of your choice.<br>*`XPlot.Plotly`* is used as per open source display package. 

In [12]:
// Plot selected bars
bars 
|> Seq.map (fun b -> (b.TimeStamp.ToDateTime(), b.Open, b.High, b.Low, b.Close))
|> Chart.Candlestick
|> Chart.WithTitle $"{sampling/60L}-min Bars for {ticker}"
|> Chart.WithXTitle "Time"
|> Chart.WithYTitle "OHLC"
|> Chart.WithWidth 1500
|> Chart.WithHeight 800
|> display

<!DOCTYPE html>